# Importación de librerías


In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
51 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as 

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
# Check if a SparkContext already exists
if not 'sc' in globals():
  sc = SparkContext()
else:
  print("Using existing SparkContext")

sqlContext = SQLContext(sc)

Using existing SparkContext


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Preprocesamiento de los datos

In [ ]:
bd = sqlContext.read.format("com.databricks.spark.csv"
).option("header", "true"
).option("inferSchema", "true"
).load("train.csv")

In [ ]:
bd.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

## Checar valores nulos en los registros


In [ ]:
from pyspark.sql.functions import col, isnan, when, count

bd.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in bd.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## Imputación de datos faltantes en age

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract, when, lit, randn
from pyspark.sql.types import IntegerType
import numpy as np
from pyspark.sql.functions import mean, stddev, udf
import random
from pyspark.sql.types import DoubleType
from pyspark.sql import functions as F

titles = ['Mr', 'Mrs', 'Master', 'Miss']

for passenger in bd:
    title = regexp_extract(col("Name"), r"([A-Za-z]+)\.", 1)
    bd = bd.withColumn("Title", when(title.isin(titles), title).otherwise(lit("Other")))


titles_count = bd.groupBy('Title').count()
print(titles_count.show())

age_stats = bd.groupBy('Title').agg(mean('Age').alias("mean_age"), stddev('Age').alias("stddev_age"))


bd = bd.join(age_stats, on="Title", how="left")

# Rellena los valores nulos en la columna "Age" con una distribución normal
bd = bd.withColumn(
    "Age",
    when(col("Age").isNull(), col("mean_age") + col("stddev_age") * randn())  # Genera un valor aleatorio basado en la media y desviación estándar
    .otherwise(col("Age"))
)

#Rellenar datos nulos con la media
mean_age = bd.select(mean('Age')).collect()[0][0]
bd = bd.na.fill(mean_age, subset=["Age"])

#Eliminar columnas adicionales
bd = bd.drop("mean_age", "stddev_age")

# Muestra el DataFrame transformado
bd.show()

+------+-----+
| Title|count|
+------+-----+
|  Miss|  182|
| Other|   27|
|Master|   40|
|    Mr|  517|
|   Mrs|  125|
+------+-----+

None
+------+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+
| Title|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+------+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+
|    Mr|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|   Mrs|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|  Miss|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|   Mrs|          4|       1|     1|Futrell

## Codificación de la variable Sex

In [ ]:
from pyspark.ml.feature import StringIndexer
# Define el indexador para la columna 'Sex'
indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")

# Ajusta el indexador a los datos y transforma el DataFrame
bd = indexer.fit(bd).transform(bd)

bd.show(5)

+-----+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+--------+
|Title|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|SexIndex|
+-----+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+--------+
|   Mr|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|     0.0|
|  Mrs|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|     1.0|
| Miss|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|     1.0|
|  Mrs|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|     1.0|
|   Mr|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          

## Codificación de la variable embarked

In [ ]:
# Rellenar valores faltantes en la columna 'Embarked' con el valor más frecuente
mode_value = bd.groupBy("Embarked").count().orderBy("count", ascending=False).first()[0]
bd = bd.na.fill({"Embarked": mode_value})

# Reemplazar valores en la columna 'Embarked'
bd = bd.withColumn(
    "Embarked",
    when(col("Embarked") == 'S', 1)
    .when(col("Embarked") == 'C', 2)
    .when(col("Embarked") == 'Q', 3)
    .otherwise(col("Embarked"))
)

# Muestra el DataFrame transformado
bd.show()

+------+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+--------+
| Title|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|SexIndex|
+------+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+--------+
|    Mr|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| NULL|       1|     0.0|
|   Mrs|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|  C85|       2|     1.0|
|  Miss|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       1|     1.0|
|   Mrs|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1| C123|    

## Eliminación de columnas innecesarias

In [ ]:
bd = bd.drop("Name", "Ticket", "Cabin", "Embarked", "Sex", "Title", "Age", "SibSp", "Parch")
bd.show(5)


+-----------+--------+------+-------+--------+
|PassengerId|Survived|Pclass|   Fare|SexIndex|
+-----------+--------+------+-------+--------+
|          1|       0|     3|   7.25|     0.0|
|          2|       1|     1|71.2833|     1.0|
|          3|       1|     3|  7.925|     1.0|
|          4|       1|     1|   53.1|     1.0|
|          5|       0|     3|   8.05|     0.0|
+-----------+--------+------+-------+--------+
only showing top 5 rows



## Normalización de datos

In [ ]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# UDF for converting column type from vector to double type
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())


for i in ["Fare","Pclass"]:

    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")


    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")


    pipeline = Pipeline(stages=[assembler, scaler])


    bd = pipeline.fit(bd).transform(bd).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")

bd = bd.drop("Fare","Pclass")
print("After Scaling :")
bd.show(5)

After Scaling :
+-----------+--------+--------+-----------+-------------+
|PassengerId|Survived|SexIndex|Fare_Scaled|Pclass_Scaled|
+-----------+--------+--------+-----------+-------------+
|          1|       0|     0.0|      0.014|          1.0|
|          2|       1|     1.0|      0.139|          0.0|
|          3|       1|     1.0|      0.015|          1.0|
|          4|       1|     1.0|      0.104|          0.0|
|          5|       0|     0.0|      0.016|          1.0|
+-----------+--------+--------+-----------+-------------+
only showing top 5 rows



# Entramiento de modelos

In [ ]:
from pyspark.sql.functions import col

# Divide el DataFrame completo en entrenamiento y prueba
train_data, test_data = bd.randomSplit([0.8, 0.2], seed=42)

# Divide el DataFrame de entrenamiento y prueba en X y Y
X_train = train_data.drop("Survived")
Y_train = train_data.select("PassengerId", "Survived")

X_test = test_data.drop("Survived")
Y_test = test_data.select("PassengerId", "Survived")

# Unir los conjuntos de entrenamiento
train = X_train.join(Y_train, on="PassengerId").drop(Y_train["PassengerId"])

# Unir los conjuntos de prueba
test = X_test.join(Y_test, on="PassengerId").drop(Y_test["PassengerId"])

test_data.show()
train_data.show()

+-----------+--------+--------+-----------+-------------+
|PassengerId|Survived|SexIndex|Fare_Scaled|Pclass_Scaled|
+-----------+--------+--------+-----------+-------------+
|          3|       1|     1.0|      0.015|          1.0|
|          7|       0|     0.0|      0.101|          0.0|
|          9|       1|     1.0|      0.022|          1.0|
|         14|       0|     0.0|      0.061|          1.0|
|         20|       1|     1.0|      0.014|          1.0|
|         24|       1|     0.0|      0.069|          0.0|
|         30|       0|     0.0|      0.015|          1.0|
|         36|       0|     0.0|      0.101|          0.0|
|         46|       0|     0.0|      0.016|          1.0|
|         47|       0|     0.0|       0.03|          1.0|
|         48|       1|     1.0|      0.015|          1.0|
|         50|       0|     1.0|      0.035|          1.0|
|         52|       0|     0.0|      0.015|          1.0|
|         56|       1|     0.0|      0.069|          0.0|
|         63| 

In [ ]:
from pyspark.ml.feature import VectorAssembler
# Crea una lista con los nombres de las columnas de características
feature_columns = [col for col in X_train.columns if col != 'PassengerId'] # Exclude 'PassengerId'

# Usa VectorAssembler para combinar las características en un solo vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Crea un DataFrame con la columna de características ensamblada
train_data = assembler.transform(train)
test_data = assembler.transform(test)


## Regresión logística

In [ ]:
from pyspark.ml.classification import LogisticRegression
# Define el modelo de regresión logística
lr = LogisticRegression(featuresCol="features", labelCol="Survived")

# Entrena el modelo
lr_model = lr.fit(train_data)

In [ ]:
# Realiza predicciones en el conjunto de prueba
predictions = lr_model.transform(test_data)

# Evalúa el rendimiento del modelo
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Define el evaluador
evaluator = BinaryClassificationEvaluator(labelCol="Survived")

# Calcula el área bajo la curva ROC
roc_auc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
print(f"Área bajo la curva ROC: {roc_auc}")

Área bajo la curva ROC: 0.8652889399158057


In [ ]:
# Mostrar el resumen del modelo
training_summary = lr_model.summary
print(f"Área bajo la curva ROC (entrenamiento): {training_summary.areaUnderROC}")
print(f"Coeficientes del modelo: {lr_model.coefficients}")
print(f"Intercepto del modelo: {lr_model.intercept}")

Área bajo la curva ROC (entrenamiento): 0.829527118316927
Coeficientes del modelo: [2.580580293330536,1.2921166719593264,-1.6301883048344652]
Intercepto del modelo: -0.5888939238500771


In [ ]:
from pyspark.sql.functions import col, when

# Calcula la matriz de confusión manualmente
tp = predictions.filter((col("Survived") == 1) & (col("prediction") == 1)).count()
tn = predictions.filter((col("Survived") == 0) & (col("prediction") == 0)).count()
fp = predictions.filter((col("Survived") == 0) & (col("prediction") == 1)).count()
fn = predictions.filter((col("Survived") == 1) & (col("prediction") == 0)).count()

print("Matriz de confusión:")
print(f"Verdaderos positivos (TP): {tp}")
print(f"Verdaderos negativos (TN): {tn}")
print(f"Falsos positivos (FP): {fp}")
print(f"Falsos negativos (FN): {fn}")

Matriz de confusión:
Verdaderos positivos (TP): 46
Verdaderos negativos (TN): 66
Falsos positivos (FP): 12
Falsos negativos (FN): 21


In [ ]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
print(f"Precisión del modelo: {accuracy}")

tpr = tp / (tp + fn)
print(f"Tasa de verdaderos positivos (TPR): {tpr}")
fpr = fp / (fp + tn)
print(f"Tasa de falsos positivos (FPR): {fpr}")

Precisión del modelo: 0.7724137931034483
Tasa de verdaderos positivos (TPR): 0.6865671641791045
Tasa de falsos positivos (FPR): 0.15384615384615385


## Random Forest


In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Modelo Random Forest
rf = RandomForestClassifier(labelCol="Survived", featuresCol="features", numTrees=100)

# Entrenar el modelo
rf_model = rf.fit(train_data)

# Hacer predicciones en el conjunto de prueba
rf_predictions = rf_model.transform(test_data)

# Evaluar el modelo de Random Forest
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
rf_accuracy = evaluator.evaluate(rf_predictions)

f1_evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="f1")
rf_f1 = f1_evaluator.evaluate(rf_predictions)

print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest F1 Score: {rf_f1}")

Random Forest Accuracy: 0.8
Random Forest F1 Score: 0.7969485060394151


## SVM

In [ ]:
from pyspark.ml.classification import LinearSVC
# Modelo SVM lineal
svm = LinearSVC(labelCol="Survived", featuresCol="features", maxIter=10)

# Entrenar el modelo
svm_model = svm.fit(train_data)

# Hacer predicciones en el conjunto de prueba
svm_predictions = svm_model.transform(test_data)

# Evaluar el modelo de SVM
svm_accuracy = evaluator.evaluate(svm_predictions)
f1_evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="f1")
svm_f1 = f1_evaluator.evaluate(svm_predictions)


print(f"SVM Accuracy: {svm_accuracy}")
print(f"SVM F1 Score: {svm_f1}")

SVM Accuracy: 0.7793103448275862
SVM F1 Score: 0.7770588471795972
